# Hands-On Workshop Exercise
---

For this exercise, your goal is to train a SAS Viya model and register it into Model Manager.

There is sample code already to load the data, transform it, train a model, and register it. However, it will fail since the model registration code is specifically for a SAS model.

You should not need to modify any other code other than the cell specified, and the first cell to specify your name.


In [1]:
name = "ssayjc"

In [2]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from pathlib import Path
import os
import requests
from sasctl import Session
from sasctl.services import model_repository as mr, model_management as mm

In [3]:
syn_credit_data = pd.read_csv("./data/generated_lendingClub1M.csv")

In [4]:
def dummy_column(column, df):
    return pd.concat([df, pd.get_dummies(df[column])], axis = 1).drop(column, axis = 1)

In [5]:
input_df = dummy_column("Purpose", syn_credit_data)


In [6]:
## Split out holdout
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    input_df.drop(['Default'], axis=1), input_df['Default'], test_size=0.3, random_state=0)

# Start Code Section
---

Edit the cell below - 

Make sure your variable for the model is called wb_model

In [7]:
wb_model = GradientBoostingClassifier()
wb_model.fit(X_train, y_train)

GradientBoostingClassifier()

# End Code Section
---
Do not edit anything after this part of the script

In [8]:
## get access token for viya env using refresh token. change to your own viya server and preferred authentication method.
url = "https://apgtps2demo.gtp.unx.sas.com"
auth_url = f"{url}/SASLogon/oauth/token"
## reading long-lived refresh token from txt file
refresh_token = Path(f'/workspaces/{os.environ["DEFAULT_MOUNTNAME"]}/apgtps2demo_refresh_token.txt').read_text().replace('\n', '')

payload=f'grant_type=refresh_token&refresh_token={refresh_token}'
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': 'Basic c2FzLmNsaTo=',
}

response = requests.request("POST", auth_url, headers=headers, data=payload, verify=False)
access_token = response.json()['access_token']

/usr/local/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apgtps2demo.gtp.unx.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [9]:
st = Session("https://apgtps2demo.gtp.unx.sas.com", token=access_token, verify_ssl=False)
st

In [10]:
project_name = "WB Hands On "+name

In [13]:
try:
    project = mr.get_project(project_name)
## Create a project
except:
    repository = mr.get_repository('DMRepository')
    # repository.name
    project_create = mr.create_project(
        {
        'name':project_name,
        'description':'WB Hands-on Exercise',
        'function':'classification',
        'targetLevel':'binary',
        },
        repository
    )
    project = mr.get_project(project_name)

In [14]:
def register_model(model_name, model, project):
    model_params = {
        "name": model_name,
        "projectId": project.id,
        "type": "ASTORE",
    }
    astore = mr.post(
        "/models",
        files={"files": (f"{model_name}.astore", model.export())},
        data=model_params,
    )
    print(astore)

#### Model Registration
---

In [15]:
register_model("MyModel "+name, wb_model, project)

MyModel ssayjc
